In [ ]:
%tensorflow_version 1.x
!pip install pydub

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random
print(tf.__version__)

1.15.2


Using TensorFlow backend.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt

In [ ]:
import json
import os
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATASET_PATH = "/content/gdrive/MyDrive/GTZAN1/Data/genres_original"
JSON_PATH = "data_11.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

In [ ]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along with genre labels.
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:

        # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment


                    # extract mfcc
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T
                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [ ]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00001.wav, segment:6
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00001.wav, segment:7
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00001.wav, segment:8
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00001.wav, segment:9
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00001.wav, segment:10
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:1
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:2
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:3
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:4
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:5
/content/gdrive/MyDrive/GTZAN1/Data/genres_original/rock/rock.00008.wav, segment:6
/content/gdrive/MyDrive/GTZAN1/Data

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

import matplotlib.pyplot as plt
import random

import librosa
import math

In [ ]:
# path to json
DATA_PATH = "/content/data_11.json"

In [ ]:
def load_data(data_path):

    with open(data_path, "r") as f:
        data = json.load(f)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [ ]:
# load data
X, y = load_data(DATA_PATH)

Data succesfully loaded!


In [ ]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# build network topology
model = tf.keras.Sequential([

    # input layer
    tf.keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    tf.keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    tf.keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    tf.keras.layers.Dense(64, activation='relu'),

    # output layer
    tf.keras.layers.Dense(10, activation='softmax')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Train on 5597 samples, validate on 2399 samples
Epoch 1/10
5597/5597 [==============================] - 3s 486us/sample - loss: 7.9017 - acc: 0.3945 - val_loss: 3.9980 - val_acc: 0.4619
Epoch 2/10
5597/5597 [==============================] - 3s 474us/sample - loss: 3.0129 - acc: 0.5146 - val_loss: 2.9225 - val_acc: 0.4885
Epoch 3/10
5597/5597 [==============================] - 3s 479us/sample - loss: 1.9634 - acc: 0.5916 - val_loss: 2.6323 - val_acc: 0.5236
Epoch 4/10
5597/5597 [==============================] - 3s 487us/sample - loss: 1.5293 - acc: 0.6391 - val_loss: 2.3725 - val_acc: 0.5402
Epoch 5/10
5597/5597 [==============================] - 3s 583us/sample - loss: 1.1514 - acc: 0.6975 - val_loss: 2.6679 - val_acc: 0.5098
Epoch 6/10
5597/5597 [==============================] - 3s 615us/sample - loss: 0.9229 - acc: 0.7402 - val_loss: 2.4059 - val_acc: 0.5369
Epoch 7/10
5597/5597 [==============================] - 3s 623us/sample - loss: 0.8322 - acc: 0.7525 - val_loss: 2.2631 - va

In [ ]:
def plot_history(history):

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")
    
    plt.show()

In [ ]:
# build network topology
model_regularized = tf.keras.Sequential([

    # input layer
    tf.keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),

    # 2nd dense layer
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),

    # 3rd dense layer
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),

    # output layer
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_regularized.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train model
history = model_regularized.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=10)

Train on 5597 samples, validate on 2399 samples
Epoch 1/10
5597/5597 [==============================] - 5s 901us/sample - loss: 23.6567 - acc: 0.2115 - val_loss: 3.5940 - val_acc: 0.3197
Epoch 2/10
5597/5597 [==============================] - 4s 725us/sample - loss: 7.5221 - acc: 0.2333 - val_loss: 3.3593 - val_acc: 0.2664
Epoch 3/10
5597/5597 [==============================] - 4s 720us/sample - loss: 4.9865 - acc: 0.2190 - val_loss: 3.4206 - val_acc: 0.1913
Epoch 4/10
5597/5597 [==============================] - 4s 685us/sample - loss: 4.1612 - acc: 0.2092 - val_loss: 3.4234 - val_acc: 0.1822
Epoch 5/10
5597/5597 [==============================] - 4s 769us/sample - loss: 3.7847 - acc: 0.2278 - val_loss: 3.4064 - val_acc: 0.1847
Epoch 6/10
5597/5597 [==============================] - 4s 778us/sample - loss: 3.6619 - acc: 0.2108 - val_loss: 3.3773 - val_acc: 0.1947
Epoch 7/10
5597/5597 [==============================] - 4s 803us/sample - loss: 3.5082 - acc: 0.2337 - val_loss: 3.3117 - v

In [ ]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [ ]:
# build the CNN
model_cnn = tf.keras.Sequential()

# 1st conv layer
model_cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(tf.keras.layers.BatchNormalization())

# 2nd conv layer
model_cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model_cnn.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(tf.keras.layers.BatchNormalization())

# 3rd conv layer
model_cnn.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu'))
model_cnn.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model_cnn.add(tf.keras.layers.BatchNormalization())

# flatten output and feed it into dense layer
model_cnn.add(tf.keras.layers.Flatten())
model_cnn.add(tf.keras.layers.Dense(64, activation='relu'))
model_cnn.add(tf.keras.layers.Dropout(0.3))

# output layer
model_cnn.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)
model_cnn.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 11, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 6, 32)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 6, 32)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 4, 32)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 2, 32)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 2, 32)         128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 1, 32)        

In [ ]:
# train model
history = model_cnn.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Train on 4797 samples, validate on 1200 samples
Epoch 1/30
4797/4797 [==============================] - 14s 3ms/sample - loss: 2.4176 - acc: 0.2487 - val_loss: 1.7721 - val_acc: 0.4150
Epoch 2/30
4797/4797 [==============================] - 10s 2ms/sample - loss: 1.6941 - acc: 0.4205 - val_loss: 1.3681 - val_acc: 0.5192
Epoch 3/30
4797/4797 [==============================] - 10s 2ms/sample - loss: 1.4322 - acc: 0.4895 - val_loss: 1.2231 - val_acc: 0.5575
Epoch 4/30
4797/4797 [==============================] - 10s 2ms/sample - loss: 1.3058 - acc: 0.5241 - val_loss: 1.1298 - val_acc: 0.5717
Epoch 5/30
4797/4797 [==============================] - 10s 2ms/sample - loss: 1.2065 - acc: 0.5601 - val_loss: 1.0866 - val_acc: 0.6050
Epoch 6/30
4797/4797 [==============================] - 10s 2ms/sample - loss: 1.1378 - acc: 0.5797 - val_loss: 1.0320 - val_acc: 0.6225
Epoch 7/30
4797/4797 [==============================] - 11s 2ms/sample - loss: 1.0796 - acc: 0.6150 - val_loss: 0.9907 - val_acc: 

In [ ]:
# evaluate model on Test Set
test_loss, test_acc = model_cnn.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

1999/1999 - 1s - loss: 0.6706 - acc: 0.7589

Test accuracy: 0.7588794


In [ ]:
model_cnn.save("Music_Genre_11_CNN")

In [ ]:
model_cnn.save("Music_Genre_11_CNN.h5")

In [ ]:
# pick a sample to predict from the test set
X_to_predict = X_test[100]
y_to_predict = y_test[100]

In [ ]:
X_to_predict.shape

(130, 13, 1)

In [ ]:
print("Real Genre:", y_to_predict)

Real Genre: 7


In [ ]:
# add a dimension to input data for sample - model.predict() expects a 4d array in this case
X_to_predict = X_to_predict[np.newaxis, ...] # array shape (1, 130, 13, 1)

In [ ]:
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
# perform prediction
prediction = model_cnn.predict(X_to_predict)

In [ ]:
# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", int(predicted_index))

Predicted Genre: 7


In [ ]:
# pick a sample to predict from the test set
X_to_predict = X_test[300]
y_to_predict = y_test[300]

print("Real Genre:", y_to_predict)

X_to_predict = X_to_predict[np.newaxis, ...]

prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", int(predicted_index))

Real Genre: 4
Predicted Genre: 2


In [ ]:
len(X_test)

1999

In [ ]:
for n in range(10):

  i = random.randint(0,len(X_test))
  # pick a sample to predict from the test set
  X_to_predict = X_test[i]
  y_to_predict = y_test[i]

  print("\nReal Genre:", y_to_predict)

  X_to_predict = X_to_predict[np.newaxis, ...]

  prediction = model_cnn.predict(X_to_predict)

  # get index with max value
  predicted_index = np.argmax(prediction, axis=1)

  print("Predicted Genre:", int(predicted_index))


Real Genre: 0
Predicted Genre: 3

Real Genre: 4
Predicted Genre: 4

Real Genre: 2
Predicted Genre: 2

Real Genre: 6
Predicted Genre: 6

Real Genre: 5
Predicted Genre: 5

Real Genre: 0
Predicted Genre: 0

Real Genre: 6
Predicted Genre: 6

Real Genre: 3
Predicted Genre: 3

Real Genre: 7
Predicted Genre: 7

Real Genre: 2
Predicted Genre: 2


In [ ]:
# Audio files pre-processing
def process_input(audio_file, track_duration):

  SAMPLE_RATE = 22050
  NUM_MFCC = 13
  N_FTT=2048
  HOP_LENGTH=512
  TRACK_DURATION = track_duration # measured in seconds
  SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
  NUM_SEGMENTS = 10

  samples_per_segment = int(SAMPLES_PER_TRACK / NUM_SEGMENTS)
  num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / HOP_LENGTH)

  signal, sample_rate = librosa.load(audio_file, sr=SAMPLE_RATE)
  
  for d in range(10):

    # calculate start and finish sample for current segment
    start = samples_per_segment * d
    finish = start + samples_per_segment

    # extract mfcc
    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=NUM_MFCC, n_fft=N_FTT, hop_length=HOP_LENGTH)
    mfcc = mfcc.T

    return mfcc

In [ ]:
genre_dict = {0:"blues",1:"rock",2:"country",3:"metal",4:"hiphop",5:"classical",6:"pop",7:"Disco"}

In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/Maroon_5_-_She_Will_Be_Loved_Offic_(getmp3.pro).mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: pop


In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/ARCH ENEMY - The Eagle Flies Alone (OFFICIAL VIDEO).mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: metal


In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/Nirvana - Smells Like Teen Spirit (Official Music Video).mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: metal


In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/Lobo Loco - The Loco Sheriff (ID 1713).mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: blues


In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/Slayer _ Angel of Death (Lyrics).mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: metal


In [ ]:
new_input_mfcc = process_input("/content/gdrive/MyDrive/Dust My Broom.mp3", 30)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [ ]:
prediction = model_cnn.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

Predicted Genre: blues
